In [1]:
# Create string with all the dates in which there is a snapshot

import datetime

start = datetime.datetime.strptime("2013-04-28", "%Y-%m-%d")
end = datetime.datetime.strptime("2018-09-09", "%Y-%m-%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days,7)]

list_of_dates = []
for date in date_generated:
    list_of_dates.append(date.strftime("%Y-%m-%d"))


In [2]:
stringlist = (','.join(list_of_dates)) 

In [3]:
dates = stringlist.replace("-","")
dates = dates.split(",")

base = 'https://coinmarketcap.com/historical/'

snaps = []
for x in dates:
       snaps.append((base + x))

In [4]:
# Load what we need

import json
import requests

# Install selenium

# In terminal: pip install selenium
#http://selenium-python.readthedocs.io/locating-elements.html

# Set up folder environment
import os
import sys
import zipfile

# Where are we?

# Great, that is the path I want
path = os.getcwd()
path

#/Users/jose/pp4rs/2018-uzh-course-material copia/14-project/Coinmarketcap scraper

# Now I want to create a folder of where to store my output
# Give it a name. Need to specify different path if you are on a Mac.
output_path = path + '\\scraper_output'

# Create a folder (if it doesnt exist yet!)
if not os.path.exists(output_path):
    os.makedirs(output_path)
    
# Great. We have an output folder

# Set up shop for scraping

# Import some useful stuff
from selenium import webdriver

# To transmit password and username
from selenium.webdriver.common.keys import Keys

# This is helpful for setting path environment to store files
from selenium.webdriver.chrome.options import Options

# To not overload the system
import time

# Define variables needed for scraping
# Set chromedriver options so that it saves the zip file in the current directory
# https://stackoverflow.com/questions/18026391/setting-chrome-preferences-w-selenium-webdriver-in-python
chromeOptions = webdriver.ChromeOptions()

# Specify where I want to store the downloads
# pass this download directory to my Chrome options
my_prefs = {"download.default_directory" : output_path}
chromeOptions.add_experimental_option("prefs", my_prefs)

# Set driver options
# Specify the path to where I stored Chromedriver
chromedriver = '/Users/jose/Desktop/DESKTOP/chromedriver'

# Start the chromedriver! 
# Specify where to find the driver and which preferences to choose
driver = webdriver.Chrome(executable_path = chromedriver, 
                          chrome_options = chromeOptions)

# Alternatively and easier is just to say without any special prefs:
#chromeOptions.add_experimental_option("prefs",prefs)

import requests
import lxml.html as lh
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs 

In [6]:
k = 0
for x in snaps:

    #Create a handle, page, to handle the contents of the website
    page = requests.get(x)

    #Make the text more bautiful:
    soup = bs(page.text, "lxml")
    
    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)

    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')

    col=[]
    i=0

    #For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
        col.append((name,[]))
    
    #Since out first row is the header, data is stored on the second row onwards
    for j in range(1,len(tr_elements)):
        #T is our j'th row
        T=tr_elements[j]
    
        #If row is not of size 11, the //tr data is not from our table 
        if len(T)!=11:
            break
    
        #i is the index of our column
        i=0
    
        #Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content() 
            #Check if row is empty
            if i>0:
            #Convert any numerical value to integers
                try:
                    data=int(data)
                except:
                    pass
            #Append the data to the empty list of the i'th column
            col[i][1].append(data)
            #Increment i for the next column
            i+=1
        
    # Create dataframe

    Dict={title:column for (title,column) in col}
    df=pd.DataFrame(Dict)

    df = df.replace('\\n', '', regex=True)

    df = df.drop(df.columns[[0,10]], axis=1) 

    # Now store the data

    import os
    os.getcwd()
    dir = os.getcwd()
    df.to_csv("/Users/jose/Desktop/PAPERS/ICO PAPER/DATA/Original scraped data/" + dates[k] + ".csv")
    k += 1

In [21]:
column_names=[]
for t in dates:
    column_names.append('Price_'+t)